## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
# 讀取酒類資料集
boston = datasets.load_boston()
boston_std_data =StandardScaler().fit_transform(boston.data)

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(boston_std_data, boston.target, test_size=0.25, random_state=42)

# 建立模型
reg = GradientBoostingRegressor(random_state=42)


In [3]:
print ("feature name=", boston.feature_names, "size=", boston_std_data)

feature name= ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT'] size= [[-0.41978194  0.28482986 -1.2879095  ... -1.45900038  0.44105193
  -1.0755623 ]
 [-0.41733926 -0.48772236 -0.59338101 ... -0.30309415  0.44105193
  -0.49243937]
 [-0.41734159 -0.48772236 -0.59338101 ... -0.30309415  0.39642699
  -1.2087274 ]
 ...
 [-0.41344658 -0.48772236  0.11573841 ...  1.17646583  0.44105193
  -0.98304761]
 [-0.40776407 -0.48772236  0.11573841 ...  1.17646583  0.4032249
  -0.86530163]
 [-0.41500016 -0.48772236  0.11573841 ...  1.17646583  0.44105193
  -0.66905833]]


In [4]:
# 先看看使用預設參數得到的結果，約為 0.14 的 MSE
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

8.908579203306832


In [5]:
from sklearn.linear_model import Lasso, Ridge
reg2 = Lasso(alpha=0.08)
reg2.fit(x_train, y_train)
y_pred = reg2.predict(x_test)
print (metrics.mean_squared_error(y_test, y_pred))
print(reg2.coef_)
reg3 = Ridge(alpha=0.01)
reg3.fit(x_train, y_train)
y_pred = reg3.predict(x_test)
print (metrics.mean_squared_error(y_test, y_pred))

22.801765374203182
[-0.84666251  0.34560047 -0.          0.69988553 -1.50411656  3.13603236
 -0.08086517 -2.33085604  0.94090733 -0.49771408 -1.87491367  1.10297899
 -3.70993256]
22.098880368923208


In [6]:
# 設定要訓練的超參數組合
n_estimators = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(reg, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1, cv = 5, iid = True)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 113 out of 120 | elapsed:    6.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    7.1s finished


In [7]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -10.926646 using {'max_depth': 3, 'n_estimators': 125}


In [8]:
grid_result.best_params_

{'max_depth': 3, 'n_estimators': 125}

In [9]:
# 使用最佳參數重新建立模型
reg_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
reg_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = reg_bestparam.predict(x_test)
# 調整參數後約可降至 0.05 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

8.62905561289985
